In [1]:
from pypokerengine.api.emulator import Emulator
from pypokerengine.players import BasePokerPlayer

# First, let's create a simple player model that always calls
class SimpleCallPlayer(BasePokerPlayer):
    def declare_action(self, valid_actions, hole_card, round_state):
        # Always call
        call_action_info = valid_actions[1]  # call is usually the second option
        return call_action_info['action'], call_action_info['amount']

    def receive_game_start_message(self, game_info):
        pass

    def receive_round_start_message(self, round_count, hole_card, seats):
        pass

    def receive_street_start_message(self, street, round_state):
        pass

    def receive_game_update_message(self, action, round_state):
        pass

    def receive_round_result_message(self, winners, hand_info, round_state):
        pass

# Set up the emulator
emulator = Emulator()
emulator.set_game_rule(player_num=2, max_round=1, small_blind_amount=5, ante_amount=0)

# Set up player information
players_info = {
    "p1": {"name": "player1", "stack": 100},
    "p2": {"name": "player2", "stack": 100},
}

# Register players
player1 = SimpleCallPlayer()
player2 = SimpleCallPlayer()
emulator.register_player("p1", player1)
emulator.register_player("p2", player2)

# Generate initial game state
initial_state = emulator.generate_initial_game_state(players_info)

# Start the round
game_state, events = emulator.start_new_round(initial_state)

# Let's see what happens in one complete round
round_state, events = emulator.run_until_round_finish(game_state)

# Print the results
print("Round finished!")
print("Final stacks:")
for player in round_state['table'].seats.players:
    print(f"{player.uuid}: {player.stack}")

Round finished!
Final stacks:
p1: 110
p2: 90


In [ ]:
from pypokerengine.api.emulator import Emulator
from pypokerengine.players import BasePokerPlayer
from pypokerengine.utils.game_state_utils import restore_game_state, attach_hole_card, attach_hole_card_from_deck

class SimpleEmulatorPlayer(BasePokerPlayer):
    def __init__(self):
        self.emulator = Emulator()
        self.emulator.set_game_rule(player_num=2, max_round=10, small_blind_amount=5, ante_amount=0)
        
    def declare_action(self, valid_actions, hole_card, round_state):
        # For each possible action, simulate what might happen
        action_results = []
        
        for action in valid_actions:
            if action['action'] in ['fold', 'call', 'raise']:
                # Create a copy of the current game state
                game_state = restore_game_state(round_state)
                
                # Attach our known hole cards
                game_state = attach_hole_card(game_state, self.uuid, hole_card)
                
                # Attach random hole cards to opponent (since we don't know them)
                for player in game_state['table'].seats.players:
                    if player.uuid != self.uuid:
                        game_state = attach_hole_card_from_deck(game_state, player.uuid)
                
                # Simulate this action to the end of the round
                amount = action['amount']
                if action['action'] == 'raise':
                    amount = action['amount']['max']  # Let's always use max raise for simplicity
                
                # Apply our chosen action
                game_state, _ = self.emulator.apply_action(game_state, action['action'], amount)
                
                # Run simulation until round ends
                game_state, _ = self.emulator.run_until_round_finish(game_state)
                
                # Get our final stack after this simulation
                final_state = game_state['table'].seats.players
                my_stack = [player.stack for player in final_state if player.uuid == self.uuid][0]
                
                action_results.append({
                    'action': action['action'],
                    'amount': amount,
                    'expected_stack': my_stack
                })
        
        # Choose the action that gives us the best expected stack
        best_action = max(action_results, key=lambda x: x['expected_stack'])
        print(f"[debug_info] Simulated results: {action_results}")
        
        return best_action['action'], best_action['amount']

    def receive_game_start_message(self, game_info):
        print(game_info)
        self.uuid = game_info['uuid']

    def receive_round_start_message(self, round_count, hole_card, seats):
        pass

    def receive_street_start_message(self, street, round_state):
        pass

    def receive_game_update_message(self, action, round_state):
        pass

    def receive_round_result_message(self, winners, hand_info, round_state):
        pass

# You can test it like this:
if __name__ == '__main__':
    from pypokerengine.api.game import setup_config, start_poker
    from pypokerengine.utils.card_utils import gen_cards
    
    config = setup_config(max_round=10, initial_stack=100, small_blind_amount=5)
    config.register_player(name="p1", algorithm=SimpleEmulatorPlayer())
    config.register_player(name="p2", algorithm=SimpleEmulatorPlayer())
    game_result = start_poker(config, verbose=1)

In [9]:
from pypokerengine.api.emulator import Emulator
from pypokerengine.players import BasePokerPlayer
from pypokerengine.utils.game_state_utils import restore_game_state, attach_hole_card, attach_hole_card_from_deck
from pypokerengine.utils.card_utils import gen_cards

class SimpleEmulatorPlayer(BasePokerPlayer):
    def __init__(self):
        self.emulator = Emulator()
        self.emulator.set_game_rule(player_num=2, max_round=10, small_blind_amount=5, ante_amount=0)
        self.uuid = None
        
    def declare_action(self, valid_actions, hole_card, round_state):
        # Convert string hole cards to Card objects
        hole_card = gen_cards(hole_card)
        
        # For each possible action, simulate what might happen
        action_results = []
        
        try:
            for action in valid_actions:
                if action['action'] in ['fold', 'call', 'raise']:
                    # Create a copy of the current game state
                    game_state = restore_game_state(round_state)
                    
                    # Attach our known hole cards
                    game_state = attach_hole_card(game_state, self.uuid, hole_card)
                    
                    # Attach random hole cards to opponent (since we don't know them)
                    for player in game_state['table'].seats.players:
                        if player.uuid != self.uuid:
                            game_state = attach_hole_card_from_deck(game_state, player.uuid)
                    
                    # Simulate this action to the end of the round
                    amount = action['amount']
                    if action['action'] == 'raise':
                        amount = action['amount']['max']  # Let's always use max raise for simplicity
                    
                    # Apply our chosen action and run simulation
                    try:
                        game_state, _ = self.emulator.apply_action(game_state, action['action'], amount)
                        game_state, _ = self.emulator.run_until_round_finish(game_state)
                        
                        # Get our final stack after this simulation
                        final_state = game_state['table'].seats.players
                        my_stack = [player.stack for player in final_state if player.uuid == self.uuid][0]
                        
                        action_results.append({
                            'action': action['action'],
                            'amount': amount,
                            'expected_stack': my_stack
                        })
                    except Exception as e:
                        print(f"Simulation failed for action {action['action']}: {str(e)}")
                        # If simulation fails, use a simple fallback strategy
                        action_results.append({
                            'action': action['action'],
                            'amount': amount,
                            'expected_stack': 0  # Pessimistic assumption
                        })
            
            # If we have any results, choose the best one
            if action_results:
                best_action = max(action_results, key=lambda x: x['expected_stack'])
                print(f"[debug_info] Simulated results: {action_results}")
                print(f"[debug_info] Choosing action: {best_action}")
                return best_action['action'], best_action['amount']
            
        except Exception as e:
            print(f"Error in declare_action: {str(e)}")
        
        # Fallback to calling if something goes wrong
        call_action = [a for a in valid_actions if a['action'] == 'call'][0]
        return call_action['action'], call_action['amount']

    def receive_game_start_message(self, game_info):
        # Find our uuid from the seats information
        for seat in game_info['seats']:
            if seat['name'] == 'p1' or seat['name'] == 'p2':  # Match either player name
                self.uuid = seat['uuid']
                break

    def receive_round_start_message(self, round_count, hole_card, seats):
        pass

    def receive_street_start_message(self, street, round_state):
        pass

    def receive_game_update_message(self, action, round_state):
        pass

    def receive_round_result_message(self, winners, hand_info, round_state):
        pass

# Test the player
if __name__ == '__main__':
    from pypokerengine.api.game import setup_config, start_poker
    
    config = setup_config(max_round=10, initial_stack=100, small_blind_amount=5)
    config.register_player(name="p1", algorithm=SimpleEmulatorPlayer())
    config.register_player(name="p2", algorithm=SimpleEmulatorPlayer())
    game_result = start_poker(config, verbose=1)

Started the round 1
Street "preflop" started. (community card = [])
Simulation failed for action call: 'ehddsweczqsaqhloowjcaq'
Simulation failed for action raise: 'ehddsweczqsaqhloowjcaq'
[debug_info] Simulated results: [{'action': 'fold', 'amount': 0, 'expected_stack': 105}, {'action': 'call', 'amount': 10, 'expected_stack': 0}, {'action': 'raise', 'amount': 100, 'expected_stack': 0}]
[debug_info] Choosing action: {'action': 'fold', 'amount': 0, 'expected_stack': 105}
"p2" declared "fold:0"
"['p1']" won the round 1 (stack = {'p1': 105, 'p2': 95})
Started the round 2
Street "preflop" started. (community card = [])
Simulation failed for action call: 'qoazihvezyxpxgojurnxmm'
Simulation failed for action raise: 'qoazihvezyxpxgojurnxmm'
[debug_info] Simulated results: [{'action': 'fold', 'amount': 0, 'expected_stack': 100}, {'action': 'call', 'amount': 10, 'expected_stack': 0}, {'action': 'raise', 'amount': 105, 'expected_stack': 0}]
[debug_info] Choosing action: {'action': 'fold', 'amoun